# The Extract Transform and Load (ETL) procedure

This notebook will walk you through the process that loads the data from a FITS cube into the Parquet tables on a Hive enables HDFS cluster. 

In [1]:
# ====================================================
# Library import
# ====================================================
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import fits
from astropy.table import Table
from astropy.visualization import astropy_mpl_style
from astropy.wcs import WCS
plt.style.use(astropy_mpl_style)

## Let's run a simple object finder on our image,
## and collect the catalog.
import numpy as np
from photutils import DAOStarFinder
from astropy.stats import sigma_clipped_stats

from scipy import sparse
from scipy.sparse import csr_matrix

from random import randint
from time import sleep

from spectral_cube import SpectralCube
from astroquery.esasky import ESASky
from astroquery.utils import TableList
from reproject import reproject_interp
import astropy.units as u

from pyspark.sql import SQLContext, SparkSession, HiveContext
from pyspark import StorageLevel

from pyspark import SparkFiles

from pyspark.sql import functions as F
import pyspark.sql.functions as f

from pyspark.sql.functions import randn, monotonically_increasing_id, row_number, desc, udf, col, lit
from pyspark.sql.functions import broadcast
from pyspark.sql.types import ArrayType, FloatType,IntegerType, DataType, DoubleType,Row, BooleanType

"""
Set the Spark datatypes
"""
iType=IntegerType()
dType=DoubleType()
fType=FloatType()

from pyspark.sql.functions import pandas_udf, PandasUDFType

from pyspark.sql.window import Window

from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import SparseVector, DenseVector,VectorUDT

from datetime import datetime
import time
import logging
import sys



maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/cache
maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/config
maindir is /home/hduser/.astropy/cache


### New Spark Context

For these processes, we need to change the spark context parameters because we're using this notebook to dynamically insert data into Hive partitioned tables; this means we need to set the hive.exec.dynamic parameters. See the context configuration below

In [2]:
conf = SparkConf().setAll([('spark.executor.memory', '8g'),\
                           ('spark.driver.memory', '6g'),\
                           ('spark.shuffle.service.enabled', True), \
                           ('spark.sql.shuffle.partitions', 144), \
                           ('spark.default.parallelism', 144),\
                           ('spark.dynamicAllocation.enabled', True), \
                           ('spark.dynamicAllocation.executorIdleTimeout', 600), \
                           ('spark.executor.cores', 4),\
                           ('spark.executor.instances', 40), \
                           ('spark.local.dir','/mnt/FITs/Spark/tmp'),\
                           ('spark.executor.memoryOverhead', '6g'),\
                           ('spark.driver.memoryOverhead', '4g'),\
                           ('spark.rpc.message.maxSize', '512'),\
                           ('spark.scheduler.mode', 'FAIR'),\
                           ('spark.kryoserializer.buffer.max', '1g'),\
                           ('spark.hadoop.hive.exec.dynamic.partition', True), \
                           ('spark.hadoop.hive.exec.dynamic.partition.mode','nonstrict'), \
                           ('spark.hadoop.hive.exec.max.dynamic.partitions', 100000), \
                           ('spark.hadoop.hive.exec.max.dynamic.partitions.pernode', 100000), \
                           ('spark.app.name','Fits on HDFS')])

    # Stop the old context
    #sc.stop()

    # And restart the context with the new configuration
    #sc = SparkContext(conf=conf)
    #sqlContext = SQLContext(sc)

In [3]:
sc.stop()

In [4]:
sc = SparkContext(conf=conf)

In [5]:
sqlContext = SQLContext(sc)
spark=SparkSession(sc)

#### Classes 

In [6]:
from time import time, clock
class Timer:
    """
    a simple class for printing time (s) since last call
    """
    def __init__(self):
        self.t0=time()
        self.t1=clock()
        self.elapsed = 1
        self.elapsedCPU = 1
    
    def start(self):
        self.t0=time()
        self.t1=clock()
        
    def stop(self):
        t1=time()
        t2=clock()
        print("Elapsed {:2.1f}s, CPU {:2.1f}s".format(t1-self.t0, t2-self.t1))
        self.elapsed = t1-self.t0
        self.elapsedCPU = t2-self.t1

class DataframeError(Exception): 
  
    # Constructor or Initializer 
    def __init__(self, value): 
        self.value = value 
  
    # __str__ is to print() the value 
    def __str__(self): 
        return(repr(self.value)) 

## Functions

The CreateAndSaveArrays function saves the positional data (in this case, Right Ascencion, Declination and Frequency) as an array element in a permanent Parquet table. the sparkfits_detail_arrays table is physically partitioned on the filename of the source FITS file and the array detail type (for example, 'RA -- SIN', 'DEC -- SIN ' and a spectra trype 'Hz', 'GHz' or 'km s-1'.

In [68]:
def CreateAndSaveArrays(array, filename, arrayCoordinates, cubeSpectraUnits=None ):
    if cubeSpectraUnits == None:
        sda_detail_name=arrayCoordinates
    else:
        sda_detail_name=cubeSpectraUnits

    #logger.in(filename, sda_detail_name )

    arrayRDD=sc.parallelize(array).zipWithIndex().map(lambda x :(x[1], x[0]) )
    arrayDF=arrayRDD.map(lambda x: (x[0], x[1].tolist()) ).toDF(["index", "spectra"])

    #"""
    #
    # Maintaining order within a collect_list, See
    # https://stackoverflow.com/questions/50766764/sorting-within-collect-list-in-hive
    #
    #"""

    arrayDF.registerTempTable("SPECTRAS")

    sqlStmt="""
    with myData as
    (
        select 1 as grp,
        index as idx,
        map(
            'spectraVal', spectra
        ) as kv
        from SPECTRAS
        distribute by grp
        sort by grp, index
    )
    select collect_list( float(a.kv['spectraVal']) ) as sda_detail_array 
    from myData a
    group by grp
    """

    spectraArrayDF=spark.sql(sqlStmt)
    spectraArrayDF=spectraArrayDF.select("sda_detail_array"\
                                         ,lit(filename).alias("sda_filename")\
                                         ,lit(sda_detail_name).alias("sda_detail_type"))

    spectraArrayDF.registerTempTable("ARRAYS")
    spark.sql("""
    insert overwrite table fits_investigation.sparkfits_detail_arrays 
    partition(sda_filename , sda_detail_type) 
    select sda_detail_array, sda_filename, sda_detail_type
    from ARRAYS
    """)



Extract and save the source FITS file header data to a Parquet table

In [8]:
def CreateFITSHeaderData(header, fitsFilename):
    hduArray=np.empty((0, 4))
    Results=Row("fits_file","index","key","value","comment")
    for i in np.arange(len(header)):
        theKey=list(header.keys())[int(i)]
        theValue=header[int(i)]
        theComment=header.comments[int(i)]

        #print(fitsFilename, theKey, theValue, theComment)
        if theKey and not theKey.isspace():
            result=Results(fitsFilename, int(i), theKey, str(theValue), header.comments[int(i)])

            if i == 0:
                hduDF=spark.createDataFrame([result])
            else:
                newRow=spark.createDataFrame([result])
                hduDF = hduDF.union(newRow)

        else:
            pass
    
    hduDF.printSchema()
    hduDF.registerTempTable("HEADERS")
    
    spark.sql("""
    insert overwrite table fits_investigation.sparkfits_fits_headers partition(sfh_fits_file ) 
    select index, key, value, comment, fits_file from HEADERS
    """)

    print('Header data created')

In [9]:
def rowdf_into_imagerdd(df, final_num_partition=1):
    """
    Reshape a DataFrame of rows into a RDD containing the full image
    in one partition.

    Parameters
    ----------
    df : DataFrame
        DataFrame of image rows.
    final_num_partition : Int
        The final number of partitions. Must be one (default) unless you
        know what you are doing.

    Returns
    ----------
    imageRDD : RDD
        RDD containing the full image in one partition
        
    NOTE: With larger FITS files, we need to avoid the use of .glom() as it results in larger memort usage.
        From the documentation -
        .glom(self) - Return an RDD created by coalescing all elements within each partition into a list.
        
    return df.rdd.zipWithIndex().coalesce(final_num_partition).glom()    
    """

    return df.rdd.zipWithIndex() #.coalesce(final_num_partition).glom()

def clean_image(im):
    """
    Cleans out the Nan elements in the array
    """
    #im=im[~np.isnan(im).any(axis=1)]
    im = np.nan_to_num(im)
    return im


def replace_nan_with_float(im):
    """
    Replaces Nan elements with a random float
    """
    
    #nan_mask = np.isnan( np.array(im, dtype=np.float32) )
    nan_mask = np.isnan( np.array(im) )
    
    sampl = np.random.uniform(low=0.01, high=13.3, size=(im.shape))
    
    im[nan_mask] = sampl[nan_mask]
    
    #random_replace = np.vectorize(lambda x: np.random.random() if np.isnan(x) else x)
    #im=random_replace(im)
    return im
    

def calculateBand(x, bandCount):
    x = x // bandCount
    return x

def CastStructToArray(StructCol):
    imageArray = np.asarray(StructCol[0]).tolist()
    return imageArray

def calculateBand(x, bandCount=32):
    x = x  // bandCount
    return x

def dummyReturn(x):
    return x.tolist()

def returnAsIs(x):
    return x

# ====================================================
# Write results to Parquet table
# ====================================================

def writeResults(sc, resultDF, vMode, vFormat, vTable):
    
    resultDF.write.mode(vMode).format(vFormat).saveAsTable(vTable)
    
def GetCubePositionData(cube, fileName, cubeDecCoordinate, cubeSpectraUnits, cubeDecDim):
    
    _, dec, _ = cube.world[0, :, 0]  #extract latitude world coordinates from cube dec
    _, _, ra = cube.world[0, 0, :]  #extract longitude world coordinates from cube ra

    """
    Make sure that we cast the Ra and Dec arrays explicitly to float32
    """
    ra=np.array(ra, dtype=np.float32)
    # and flip - always want declination ordered from greatest to lowest. eg -18 ..... -27 or 43 ..... 21
    #if max(dec) <0 or 1==1:
    
    dec=np.flip( np.array(dec, dtype=np.float32) )
    
    #else:
    #    dec=np.array(dec, dtype=np.float32)
    
    """
    
    Get the Spectral Axis data.
    Note: Further enhancements can include changing the values of the Spectral axis - ie change from Hz to redshift or
    velocity.
    
    """
    spectralAxis=cube.spectral_axis
    
    """
    Cast the SpectralCube explicitily to a numpy array, float 32
    """
    
    specArray=np.array(spectralAxis , dtype=np.float32)
    
    print("arrays created")
    """ parallelize the arrays 
    
    NOTE: 
    As spark does not necessarily maintain the order of the values in a join, we need to specifically define the ordering
    in order to maintain the correct relationships between the position and spectra arrays we extract from te FITS cube via 
    SpectralCube during the population process.
    
    To do this, we create a parallelised array for the spectra and declinatino with a zip index. the cartesian join is then sorted 
    spectra and declination indexes, and then the index is created for the cartesian RDD.
    
    This ensures the proper sorting and ordering of the combined data.
    
    Note that this is a potentially an expensive computational operation on larget arrays, but as it is only a one-off
    computation per file, this is acceptable.
    """
    rdd1=sc.parallelize(specArray).zipWithIndex()
    rdd2=sc.parallelize(dec).zipWithIndex()
    
    print("and parallelized")
    
    """ Ordered cartesian join as a basis of the dataframe """
    fDecDF=rdd1.cartesian(rdd2)\
    .sortBy(lambda x:(x[0][1], x[1][1]), True)\
    .zipWithIndex()\
    .map(lambda x: (x[0][0][0].tolist(), x[0][1][0].tolist(), x[1]))\
    .toDF(["spd_spectra_value","spd_declination", "spd_index"])\
    .select("spd_index", \
                castToFloat("spd_spectra_value").alias("spd_spectra_value"), \
                castToFloat("spd_declination").alias("spd_declination"), \
                lit(fileName).alias("spd_filename"), \
                lit(cubeDecCoordinate).alias("spd_position_type"), \
                lit(cubeSpectraUnits).alias("spd_spectra_type"), \
                GetBandUDF("spd_index", lit(cubeDecDim)).alias("spd_band")  )                
    print("dataframe created")
    return ra, dec, specArray, fDecDF


## UDF Wrapper functions

In [10]:
GetBandUDF = udf(lambda arr, bw: calculateBand(np.int(arr), bw), iType)
CastToInt = udf(lambda x: dummyReturn(x), iType)
CastToFloatArrayUDF = udf(lambda x : CastStructToArray(x), ArrayType(fType))

castToFloat = udf(lambda arr: returnAsIs(arr), fType)


In [11]:
sqlContext.sql("use fits_investigation")

DataFrame[]

In [12]:
sqlContext.sql("show tables").show()

+------------------+--------------------+-----------+
|          database|           tableName|isTemporary|
+------------------+--------------------+-----------+
|fits_investigation|          array_test|      false|
|fits_investigation|    fits_header_data|      false|
|fits_investigation|          float_test|      false|
|fits_investigation|  imagetestsparkfits|      false|
|fits_investigation|   localparalleltest|      false|
|fits_investigation|localparalleltest...|      false|
|fits_investigation|localparalleltest...|      false|
|fits_investigation|localparalleltest...|      false|
|fits_investigation|  newdfdeletemelater|      false|
|fits_investigation|     parquetdataview|      false|
|fits_investigation|parquetpartitione...|      false|
|fits_investigation|parquetpartitione...|      false|
|fits_investigation|parquetpartitione...|      false|
|fits_investigation|parquetpartitione...|      false|
|fits_investigation|parquetpartitione...|      false|
|fits_investigation|parquetp

In [36]:
UseResidual=False

if UseResidual:
    bigFilePath='hdfs:///user/hduser/FITS_Files/residual.i.SB8170.cube.fits'
    bigFileName="residual.i.SB8170.cube.fits"
    localBigFile="/mnt/FITs/residual.i.SB8170.cube.fits"
else:
    bigFilePath='hdfs:///user/hduser/FITS_Files/image.restored.i.SB2338.V2.cube.fits'
    bigFileName="image.restored.i.SB2338.V2.cube.fits"
    localBigFile="/mnt1/FITs/image.restored.i.SB2338.V2.cube.fits"





Create the dataframe 

In [37]:
timer=Timer()
hdu=0
df = sqlContext.read.format("fits").option("hdu", hdu).load(bigFilePath)
fits.info(localBigFile)
hdu1=fits.open(localBigFile , memmap=True)
cubeHeader=hdu1[0].header
cubeWCS=WCS(cubeHeader)
cubeHz = SpectralCube.read(hdu1)

hdu1.close()
PartitionFileName = bigFileName

Filename: /mnt1/FITs/image.restored.i.SB2338.V2.cube.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      77   (5607, 5654, 1, 2592)   float32   


Save the header data

In [40]:
CreateFITSHeaderData(cubeHeader, PartitionFileName)

root
 |-- fits_file: string (nullable = true)
 |-- index: long (nullable = true)
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- comment: string (nullable = true)

Header data created


In [44]:
PartitionFileName

'image.restored.i.SB2338.V2.cube.fits'

In [45]:
polarisation=cubeHeader["NAXIS3"] # Polarisation 196608
raLength=cubeHz.header["NAXIS1"] # RA
decLength=cubeHz.header["NAXIS2"] # DEC
spectraLength=cubeHz.header["NAXIS3"] # Spectra (hx) Pixels 

numPolarisation=cubeHeader["NAXIS4"]

## Same as dec length, used in the GetBandUDF UDF functionsto calculate the polarisation and spectral band indexes.
## See
##        GetBandUDF(col("spi_index"), lit(frequencyBandLength*spectraLength)).alias("spi_pol"),\
##        GetBandUDF(col("spi_index"), lit(frequencyBandLength)).alias("spi_band") )\


frequencyBandLength=cubeHz.header["NAXIS2"] 


In [46]:
polarisation

1

Get the Sparkfits dataframe, and then populate the images table with the dataframe

In [47]:
timer.start()
partitionTarget=df.rdd.getNumPartitions()
print("Source file partition numbers - {}".format(partitionTarget))

### Create the RDD with the zip index
imageRDD=rowdf_into_imagerdd(df)

print("RDD with zip index created")
timer.stop()

timer.start()
bRemoveNaNs=False

if bRemoveNaNs:
    print("Replacing NaN values with random float")
    PartitionFileName=PartitionFileName+'.NaNs.Removed'
    imageDF=imageRDD.map(lambda x: ( replace_nan_with_float(np.array(x[0]) ).tolist(), x[1] ) ).toDF(["spi_image","spi_index"])
else:
    imageDF=imageRDD.toDF(["spi_image","spi_index"])


numPartitions = imageDF.rdd.getNumPartitions()

if numPartitions < partitionTarget:
    imageDF=imageDF.repartition(partitionTarget)
    print("Repartitioning the dataframe from {} to {} partitions...".format(numPartitions, partitionTarget))

dff=imageDF.select(\
                   "spi_index", \
                   CastToFloatArrayUDF("spi_image").alias("spi_image"), \
                   lit(PartitionFileName).alias("spi_filename"), \
                   GetBandUDF("spi_index", lit(frequencyBandLength*spectraLength)).alias("spi_pol"),\
                   GetBandUDF("spi_index", lit(frequencyBandLength)).alias("spi_band") )\
.persist(StorageLevel.MEMORY_AND_DISK)

print("Dataframe created and persisted")

timer.stop()

Source file partition numbers - 2449
RDD with zip index created
Elapsed 542.2s, CPU 0.3s
Dataframe created and persisted
Elapsed 0.4s, CPU 0.0s


In [48]:
dff.printSchema()

root
 |-- spi_index: long (nullable = true)
 |-- spi_image: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- spi_filename: string (nullable = false)
 |-- spi_pol: integer (nullable = true)
 |-- spi_band: integer (nullable = true)



In [49]:
dff.registerTempTable("IMAGES")

In [51]:
spark.sql("select spi_index, spi_image, spi_filename, spi_band from IMAGES").show(20)

+---------+--------------------+--------------------+--------+
|spi_index|           spi_image|        spi_filename|spi_band|
+---------+--------------------+--------------------+--------+
|        0|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|        1|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|        2|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|        3|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|        4|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|        5|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|        6|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|        7|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|        8|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|        9|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|       10|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|       11|[0.0, 0.0, 0.0, 0...|image.restored.i....|       0|
|       12|[0.0, 0.0, 0.0, 0...|image.restored.i....|  

Write the dataframe into the target table.

Check the polarisations. If there is only one polarisation, we can ignore that and insert into the sparkfits_images table; otherwise we insert into sparkfits_polarised_images.

In [53]:
print("Writing dataframe to parquet images table")
timer.start()
if polarisation==1:
    print("Saving to sparkfits_images")
    spark.sql("""
    insert overwrite table fits_investigation.sparkfits_images partition(spi_filename, spi_band ) 
    select spi_index, spi_image, spi_filename, spi_band from IMAGES
    """)
else:
    print("Saving to sparkfits_polarised_images")
    spark.sql("""
    insert overwrite table fits_investigation.sparkfits_polarised_images partition(spi_filename, spi_pol, spi_band ) 
    select spi_index, spi_image, spi_filename, spi_pol, spi_band from IMAGES
    """)

print("All done!")
timer.stop()

Writing dataframe to parquet images table
Saving to sparkfits_images
All done!
Elapsed 1749.8s, CPU 0.5s


## Dimension Table Population

The example below create dimension tables for cubes with differing spectral units. We create two new SpectralCube objects with velocity and GHz spectra,

In [54]:
print("Populating the dimension tables")

timer.start()
print("Create the alternative spectral cubes for the differing spectral arrays")
cubeVel = cubeHz.with_spectral_unit(u.km / u.s, velocity_convention='radio' ,rest_value=200 * u.Hz)
cubeGHz = cubeHz.with_spectral_unit(u.GHz, velocity_convention='radio' ,rest_value=200 * u.GHz)
print("Alternative cubes created")
timer.stop()


Populating the dimension tables
Create the alternative spectral cubes for the differing spectral arrays
Alternative cubes created
Elapsed 0.1s, CPU 0.1s


In [ ]:
for i, c in enumerate([cubeHz, cubeVel, cubeGHz]):
    timer.start()
    print(c.header["ORIGIN"])

    cubePixelValue = c.header["BUNIT"]
    cubeRaDim = c.header["NAXIS1"]
    cubeDecDim = c.header["NAXIS2"]
    cubeSpectraDim = c.header["NAXIS3"]
    cubeRACoordinate = c.header["CTYPE1"]
    cubeDecCoordinate = c.header["CTYPE2"]
    cubeSpectraCoordinate = c.header["CTYPE3"]
    cubeRAUnits = c.header["CUNIT1"]
    cubeDecUnits = c.header["CUNIT2"]
    cubeSpectraUnits = c.header["CUNIT3"]
    print(cubePixelValue, cubeSpectraCoordinate, cubeSpectraUnits)
    timer.start()

    ra, dec, specArray, DecDF = GetCubePositionData(c, PartitionFileName, cubeDecCoordinate, cubeSpectraUnits, cubeDecDim)

    """
    Add the declination and frequency dimension to the table
    """
    print("Creating the temp table POSDIM ")

    DecDF.registerTempTable("POSDIM")
    
    newPosDimDF=spark.sql("""
        with myData as
        (
        select
        spd_filename,spd_position_type,spd_spectra_type,
        spd_declination,
            map(
                'idxVal', spd_index
            ) as kv
            from POSDIM
            distribute by spd_declination
            sort by spd_index
        )
        select
        spd_filename,spd_position_type,spd_spectra_type,
        spd_declination,
        collect_list( float(a.kv['idxVal']) ) as spd_declination_spectra 
        from myData a
        group by spd_filename,spd_position_type,spd_spectra_type,spd_declination
        order by spd_declination desc

    """)

    newPosDimDF.registerTempTable('NewPosDim')

    spark.sql("""
    insert overwrite table fits_investigation.sparkfits_new_position_dimension
    partition(spd_filename, spd_position_type, spd_spectra_type) 
    select spd_declination,spd_declination_spectra,
    spd_filename,spd_position_type,spd_spectra_type
    from NewPosDim
    """)
    
    if False:
        spark.sql("""
        insert overwrite table fits_investigation.sparkfits_position_dimension 
        partition(spd_filename, spd_position_type, spd_spectra_type, spd_band) 
        select spd_index, spd_spectra_value, spd_declination,
        spd_filename,spd_position_type,spd_spectra_type,spd_band
        from POSDIM
        """)

    if i == 0:
        print("Creating ra and dec detail arrays")
        CreateAndSaveArrays(ra,  PartitionFileName, cubeRACoordinate)
        CreateAndSaveArrays(dec, PartitionFileName, cubeDecCoordinate)

    print("Creating the spectral detail array")
    CreateAndSaveArrays(specArray, PartitionFileName, cubeSpectraCoordinate, cubeSpectraUnits) 
    timer.stop()